In [42]:
from IPython.display import display, HTML           # updated following <https://github.com/googleapis/python-aiplatform/issues/5079>
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk import word_tokenize, bigrams, trigrams, pos_tag


import re

In [44]:
# fix annoying pandas display restrictions
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', None)


- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [45]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system 
#data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(5964, 2)


### Let's divide the training and test set into two partitions

I am very confused as to why we would want to do this *before* the data preprocessing?! 
Also, there is a kg_test.csv already, which we should probably use.

I'll go ahead with the cleaning first. Also, it would also be much more advantageous to write a function for the cleaning rather than a piecemeal declarative recipe, since we'll have to run this over the test set again later!

In [46]:
# Your code
data.head()

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL I AM MIKE CHUKWU , THE MANAGER, BILLS AND EXCHANGE AT THE FOREIGN REMITTANCE DEPARTMENT OF THE ZENITH INTERNATIONAL BANK PLC. I AM WRITING THIS LETTER TO ASK FOR YOUR SUPPORT AND COOPERATION TO CARRY OUT THIS BUSINESS OPPORTUNITY IN MY DEPARTMENT. WE DISCOVERED AN ABANDONED SUM OF $15,000,000.00 (FIFTEEN MILLION UNITED STATES DOLLARS ONLY) IN AN ACCOUNT THAT BELONGS TO ONE OF OUR FOREIGN CUSTOMERS WHO DIED ALONG WITH HIS ENTIRE FAMILY OF A WIFE AND TWO CHILDREN IN NOVEMBER 1997 IN A PLANE CRASH. SINCE WE HEARD OF HIS DEATH, WE HAVE BEEN EXPECTING HIS NEXT-OF-KIN TO COME OVER AND PUT CLAIMS FOR HIS MONEY AS THE HEIR,BECAUSE WE CANNOT RELEASE THE FUND FROM HIS ACCOUNT UNLESS SOMEONE APPLIES FOR CLAIM AS THE NEXT-OF-KIN TO THE DECEASED AS INDICATED IN OUR BANKING GUIDELINES. UNFORTUNATELY, NEITHER THEIR FAMILY MEMBER NOR DISTANT RELATIVE HAS EVER APPEARED TO CLAIM THE SAID FUND. UPON THIS DISCOVERY,I AND OTHER OFFICIALS IN MY DEPARTMENT HAVE AGREED TO MAKE BUSINESS WITH YOU AND RELEASE THE TOTAL AMOUNT INTO YOUR ACCOUNT AS THE HEIR OF THE FUND SINCE NO ONE CAME FOR IT OR DISCOVERED HE MAINTAINED ACCOUNT WITH OUR BANK, OTHERWISE THE FUND WILL BE RETURNED TO THE BANKS TREASURY AS UNCLAIMED FUND. WE HAVE AGREED THAT OUR RATIO OF SHARING WILL BE AS STATED THUS; 30 % FOR YOU AS FOREIGN PARTNER, 60 % FOR US THE OFFICIALS IN MY DEPARTMENT AND 10% FOR THE SETTLEMENT OF ALL LOCAL AND FOREIGN EXPENCES INCURRED BY US AND YOU DURING THE COURSE OF THIS BUSINESS. UPON THE SUCCESSFUL COMPLETION OF THIS TRANSFER, I AND ONE OF MY COLLEAGUES WILL COME TO YOUR COUNTRY AND MIND OUR SHARE. IT IS FROM OUR 60 % WE INTEND TO IMPORT AGRICULTURAL MACHINERIES INTO MY COUNTRY AS A WAY OF RECYCLING THE FUND. TO COMMENCE THIS TRANSACTION, WE REQUIRE YOU TO IMMEDIATELY INDICATE YOUR INTEREST BY A RETURN E-MAIL AND ENCLOSE YOUR PRIVATE CONTACT TELEPHONE NUMBER, FAX NUMBER FULL NAME AND ADDRESS AND YOUR DESIGNATED BANK COORDINATES TO ENABLE US FILE LETTER OF CLAIM TO THE APPROPRIATE DEPARTMENTS FOR NECESSARY APPROVALS BEFORE THE TRANSFER CAN BE MADE. NOTE ALSO, THIS TRANSACTION MUST BE KEPT STRICTLY CONFIDENTIAL BECAUSE OF IT'S NATURE. I LOOK FORWARD TO RECEIVING YOUR PROMPT RESPONSE. REGARDS, MR. MIKE CHUKWU ZENITH INTERNATIONAL BANK PLC .",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about Haiti to me this weekend. Can you please print them and organize them inchrono order for me for my trip tomorrow or send to Lauren so she can do? Thx.,0
3,Dear Sir=2FMadam=2C I know that this proposal might be a surprise to you but it as an emergency=2EIn a nutshell=2C I am MR=2EP=2E ZUMA=2C from the republic of South-Africa=2C now seeking for refuge in Dakar Senegal under the=28UNHCR=29=2E I got your contact during a desperate search here in Dakar for a possible transaction=2E My =28late=29 father DR=2E ZUMA KENT WILLIAMS=2C he was the managing director of Gold Mine company in SOUTH AFRICA=2E But he was assinated by his business assoicate and all his properties was totally destroyed by his elder brother=2E Who alway hate to see me and my mother with his eye's=2C that man very wicked that is why i and my mother managed to escape with some of my father's documents covering $18=2E3 Million dollars which is presently deposited safely in a finance firm on my name as the next of kin=2E Meanwhile=2Cwe are saddled with the problem of securing a trust-worthy foriegn personality to help us transfer the money over to his country and into his possession pending our arrival to meet with him=2E Furthermore=2Cwe only want this done this way because your country is politically stable for any profitable investment and only if you accept our proposal=2C you will serve as the beneficiary of the fund on commencement of this proposed transactions=2E I am giving you the offers as mentioned with every confidence on your acceptance to assist us=2C we have decided to invest 40% of the total fund based on equity participation

In [47]:
test = pd.read_csv("../data/kg_test.csv",encoding='latin-1')

test.head()

,text
0,"usiness is for the fact that the deceased man was a foreigner and it is not= authorized by the law guiding our Bank for a citizen of this country to ma=ke the claim of the fund.So the request for you as a foreigner to come into= the business is necessary, it will facilitate the release and transfer of =the fund smoothly into your reliable Bank account as the only surviving rel=ation to the deceased.<BR><BR>If you are really sure of your integrity, tru=stworthy and confidentiality, and accept not to cheat or disappoint me afte=r this business, reply with your telephone and fax numbers and Banking acco=unt details where you wish to receive the fund.<BR><BR>Upon receipt of the =above mentioned information, indicating your interest in the business, I wi=ll send you by fax or e-mail a letter of application which you will type an=d send back to our bank to enable us start this transaction.<BR><BR>I will =not fail to bring to your notice that this business is risk free and doesn'=t have any negative implication. You should not entertain any fear as all m=odalities for the smooth and easy transfer of this fund have been finalized=. This transaction will be completed within 14 bank working days immediatel=y we start the business.<BR><BR>Waiting for your urgent response and please= kindly delete if you are not interested.<BR>Regards,<BR><BR>&nbsp;Mr&nbsp;="
1,They are happy to adjust to the afternoon. I am going to suggest a 3:00pm start time which should give plenty of time toreturn to DC. (Cronkite service starts at 10:30am)
2,Lael Brainard was confirmed 78-19 this afternoon.Miguel E. RodriguezDeputy Assistant Secretary for Senate AffairsBureau of Legislative AffairsU.S. Department of State(202) 647-2645rodriguezme@state.gov
3,H <hrod17@clintonemail.com>Friday March 26 2010 5:45 AM'sbwhoeopÃÂ¢ÃÂÃÂ¢ Re:I have extended my congrats to all.
4,"n;""> Dear Good Friend,<br><br><br>I am happy to inform you about my succe=ssin getting those funds transferred under<br><br>the cooperation of a ne=w partner from Hongkong.Presently I am in Hongkong for investment project=s<br><br>with my own share of the total sum. meanwhile, Ididn't forget yo=ur past efforts and attempts to assist<br><br>me in transferring those fu=nds despite that it failedus some how.<br><br><br>Now contact my secretar=y in Cotonou Benin Republic hisname is MR SHIKES UMAHand his Email:(secre=taryshikesuperstar@yahoo.ca)<br><br> <br><br> and ask him to send you the= total of $800.000.00 which I kept for your<br><br>past efforts and attem=pts toassist me in this matter.I appreciated your efforts atthat time ver=y much.<br><br>so feel free and get intouchedwith my SecretarySHIKES UMAH= and instruct him where to sendthe amount to you.<br><br><br>Please do le=t me know immediately you receive it sothat we can share the joy after al=l the sufferness at<br><br>that time. in the moment, I am very busy here =becauseof the investment projects which me and the new<br><br>partner are= having at hand, finally, remember that Ihad forwarded instruction to the= secretary on your<br><br>behalf to receive that money, so feel free to g=et intouch with Mr SHIKES UMAH he will send the amount to you with out =any delay.<br><br><br>Thanks and God bless you.<br><br>SHIKES UMAH </sp=an><br><br>- <span style=3D""color:green;"">mikejames2 james</span></div></=div>--0-1398371323-1145545952=:82031--"


## Data Preprocessing

In [48]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


In [49]:
from bs4 import BeautifulSoup

## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [50]:
# Function to remove tags
def remove_tags(html):

    # parse html content
    soup = BeautifulSoup(html, "html.parser")

    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()

    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)


In [51]:
# check if the remove_tags function is performing as intended
htmltest=r"<script>ingenious java script</script> and <b>this</b> is <a href='http://greatlink.com'>a great link outside</a> the script"
htmltest2=r"what happens <!--with html comments-->, since the task is asking about this in <i>particular</i>"
htmltest_nohtml=r"This is just some plain boring text"

remove_tags(htmltest_nohtml)

'This is just some plain boring text'

In [52]:
# Your code
data_dehtml = data.copy()
test_dehtml = test.copy()
data_dehtml['text'] = data_dehtml.text.apply(remove_tags)
test_dehtml['text'] = test_dehtml.text.apply(remove_tags)

/tmp/ipykernel_300410/1293059053.py:5: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  soup = BeautifulSoup(html, "html.parser")


In [53]:
data_dehtml.tail()

,text,label
5959,I talked to CDM about doing some strategy sessions this week. We discussed trying to do three:There are obviously a dozen different ways to do this and a dozen different configurations. So consider this a strawman- we're eager to hear your reactions.,0
5960,"Dear Sir/CEO,=20I am Mr.D S Ammer, the purchasing director African Millennium Villages Pr=oject. Our head office is situated in Lome, Federal Republic Togo. This a=gency has been existing since over 3 years.=20Our duty is to fight against rural poverty in Africa. This scheme has ove=r US$500,000,000,00 involves targeted investment in health, education and= farming to help lift African villages out of poverty.Presently we have currently covers around 107 villages in 13 countries, i=ncluding, Ethiopia, Ghana, Mali, Rwanda, Uganda and Kenya. Under the sche=me, villages are supplied with various materials, e.g. mosquito nets to p=revent the spread of malaria, fertilizers, clothing of various kinds and =treatment for people living with HIV/Aids. This scheme makes up part of t=he UN'S Millennium Development Goals, which are internationally agreed ta=rgets to cut poverty, hunger, disease and boots literacy by 2015 in Afric=a.=20I am contacting you for a business; I would like you to register with our= agency as one of our foreign suppliers that supplys relief materials to =our organization. I have about US$8MILLION to transfer into any secured b=ank account abroad. The money is an over-invoiced contract amount execute=d by a foreign supplier and commissioned with African Millennium Villages= project (AMVP), and the original supplier has been paid since march last= year remaining the over-invoiced amount of $8million. This was intention=al and since then the US$8million has been in the suspense account of our= organization awaiting for claim. Now i am ready to transfer the fund(US$=8million) overseas for our personal business investment and that is where= i need your assistance by providing me with a trusted bank account to re=mit the fund. Then both of us will share the fund.=20Now, how to start is just for me to pass through my attorney and register= your name/company as a sub-contract with African Millennium Villages pro=ject, and the date of the registration will be back-dated to the date the= contract was awarded to the original supplier. Then you will serve among= the foreign suppliers that supplied relief material to our organization =awaiting for payment.=20All arrangements have been put in place to ensure a successful project. P=lease if you are capable of working this project with me get back to me q=uickly via this email:deansammer033@yahoo.com. We will discuss on the sha=ring modality of the fund if i receive your positive response. I will sen=d you more details on how we are going to proceed on the claim. Note this= project is risk free and genue.With your corporation it will only take j=ust few weeks to accomplish our project.=20Kind regards=20Mr.D S Ammer=20",1
5961,Faxing to you now.,0
5962,"Goodday,I am Joseph vaye,the son of late Issac Nuhan Vaye, deputy minister of publicworks under President charles taylor of Liberia.Before the political upheaval in liberia my late father was a good andclose friend of presidentcharles taylor and a member of his inner caucus.But of late when liberiacrisis came up again,with the rebels trying to oust charles taylor outof government and liberia,president charles taylor became very uncomfortablewith some of his loyalists which my late father happened to be one of thembecause he had the believe that at the rate at which the rebel group was closing in on him in Monrovia(which is the seat of power)was as a resultof the leakage of some vital security information from his cabinet and hedecided to eliminate some of his cabinet members which my father happenedto be a victim.It started in the name of an uncovered coup and he ordered the arrest ofsome of his cabinet ministersand the Vice President Moses Blah on the 5th of j

Some mails (mabye particularly the spam ones?) have missing spaces all over the place (e.g. in 996). Some of them might be fixed by checking for lower case characters directly followed by an upper case character (e.g. "inLondon"), or by a string of letters directly following punctuation ("inLondon.I feel..."). Many other cases would only be retrievable with some dictionary lookup, e.g. in wordnet (e.g. "isunofficial"). Not sure if it's worth the effort for the latter, but we could at least fix the first two patterns using re directly, before we get rid of the punctuation and lose this incredible business opportunity...

In [54]:
missingspace = re.compile(r'([\.]|(?:[A-Za-z]*[a-z]))([A-Z][A-Za-z]*)')

In [55]:
trial = """TRANSFER  .This house OF 36,759,000.00 MILLION POUNDS TO YOURACCOUNT.My name is Mr.EJIS DON and 
I work in theInternational Operation Department in a Bank here inLondon.I feel quite safe dealing with you in thisimportant business.Though, this medium 
(Internet) hasbeen greatly
"""

missingspace.sub(r'\1 \2',trial)

'TRANSFER  . This house OF 36,759,000.00 MILLION POUNDS TO YOURACCOUNT. My name is Mr. EJIS DON and \nI work in the International Operation Department in a Bank here in London. I feel quite safe dealing with you in thisimportant business. Though, this medium \n(Internet) hasbeen greatly\n'

In [56]:
data_dehtml['text'] = data_dehtml.text.apply(lambda x: missingspace.sub(r'\1 \2',x))
test_dehtml['text'] = test_dehtml.text.apply(lambda x: missingspace.sub(r'\1 \2',x))

There are some HTML remnants of the form '=2F' and similar, presumably hex encodings for ascii signs. Convert them with another function. (To be honest, this might be a bit pointless, since we'll be removing special chars later, but I still feel this is a more systematic approach. Alternatively, I suppose we could just sub all of these with zero strings.)

In [57]:
# function to turn remnants of html encoding (e.g. '=2F') into the corresponding characters, assuming ASCII encoding
def html_remnant_to_char(inputstring):
    return chr(int(inputstring.replace('=',''),16))

# regular expressions to find html remnants
htmlrem = re.compile(r'=[2-9A-F][0-9A-F]')

def fix_html_remnants(inputstring,pattern):
    return pattern.sub(lambda x: html_remnant_to_char(str(x.group())),inputstring)
    

In [58]:
s = '2F'
chr(int(s,16))

'/'

In [59]:

test= r'Dear Sir=2FMadam=2C I know that this proposal might be a surprise to you but it as an emergency=2EIn a nutshell=2C I am MR=2EP=2E ZUMA=2C from the republic of South-Africa=2C now seeking for refuge in Dakar Senegal under the=28UNHCR=29=2E  '
htmlrem.sub(lambda x: html_remnant_to_char(str(x.group())),test)



'Dear Sir/Madam, I know that this proposal might be a surprise to you but it as an emergency.In a nutshell, I am MR.P. ZUMA, from the republic of South-Africa, now seeking for refuge in Dakar Senegal under the(UNHCR).  '

In [60]:
fix_html_remnants(test,htmlrem)

'Dear Sir/Madam, I know that this proposal might be a surprise to you but it as an emergency.In a nutshell, I am MR.P. ZUMA, from the republic of South-Africa, now seeking for refuge in Dakar Senegal under the(UNHCR).  '

In [61]:
# apply this substitution to train and test sets

data_dehtml['text'] = data_dehtml.text.apply(fix_html_remnants,pattern=htmlrem)
test_dehtml['text'] = test_dehtml.text.apply(fix_html_remnants,pattern=htmlrem)

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [62]:
data_dehtml.iloc[298]

text     http://www.ft.com/cms/s/5799a774-b534-11df-9af8-00144feabdc0dwp_uuid#6ce648-33fa-11da-adae-00000e2511c8print=yes.html Financial Times FT.comr. OLUMNISTSJVlartin Wolf Obama was too cautious in fearful times By Martin Wolf Published: AuÃ¢ÂÂ¢ ust 31 2010 22:48 j Last updated: August 31 2010 22:48Suppose that the US presidential election of 1932 had in fact taken place in 1930 at an early stage in the Great Depression. Suppose too that Franklin Delano Roosevelt had won then though not by the landslide of 1932. Howdifferent subsequent events might have been. The president might have watched helplessly as output and employmentcollapsed. The decades of Democratic dominance might not have happened. On such chances the wheel of history turns. But this time was different: the crisis brought Barack Obama to power closeto the beginning of the economic collapse. I (among others) then argued that policy needed to be hugely aggressive. Alasit was not. I noted on February 4 2009 at the begin

Hm, removing all single characters also removes I. But I guess for for the current task pronouns are considered "stop words" anyway, so probably not an issue.

In [63]:
# remove emails
email = re.compile(r'\w+(?:\.\w+)?@\w+\.[a-zA-Z]{2,3}(?:\.[a-zA-Z]{2,3})?')
website = re.compile(r'http[s]?://\w+(?:\.\w+){2,3}(?:/[\w+-=&%#]+)*[/]?')
single_letter = re.compile(r'\b([A-Za-z])\b')
non_letter = re.compile(r'[^A-Za-z\s]')

testmail = 'haus ist blasfj_dsf@rte74.cde.ed und so weiter'
testwebsite = 'dsad@sds.de.ef http://www.ft.com/cms/s/5799a774-b534-11df-9af8-00144fea#bdc0dwp_uuid=236ce648-33fa-11da-adae-00000e2511c8print=yes.htmlFinancial Times F'
print(email.findall(testmail))
print(website.sub(' ',testwebsite))

['blasfj_dsf@rte74.cde.ed']
dsad@sds.de.ef   Times F


In [64]:
# now let's get wild and delete all emails, websites and finally all non-letters

deletelist = [website,email,single_letter,non_letter]

for p in deletelist:
    data_dehtml['text'] = data_dehtml.text.apply(lambda x: p.sub('',x))
    test_dehtml['text'] = test_dehtml.text.apply(lambda x: p.sub('',x))



In [65]:
# check removal of hyperlink
data_dehtml.iloc[298]

text      Financial Times FTcomr OLUMNISTSJVlartin Wolf Obama was too cautious in fearful times By Martin Wolf Published Au ust     Last updated August   Suppose that the US presidential election of  had in fact taken place in  at an early stage in the Great Depression Suppose too that Franklin Delano Roosevelt had won then though not by the landslide of  Howdifferent subsequent events might have been The president might have watched helplessly as output and employmentcollapsed The decades of Democratic dominance might not have happened On such chances the wheel of history turns But this time was different the crisis brought Barack Obama to power closeto the beginning of the economic collapse  among others then argued that policy needed to be hugely aggressive Alasit was not  noted on February   at the beginning of the new presidency Instead of an overwhelming fiscalstimulus what is emerging is too small too wasteful and too illfocused  week later  asked Has Barack Obamaspresidency alr

In [66]:
# remove punctuation and special signs
punct = re.compile(f"[{re.escape(string.punctuation)}]")
compressspace = re.compile(r'\s+')                  # probably could be taken care of by tokenisation later anyway, but let's still do it

replacebysinglespace = [punct, compressspace]

for p in replacebysinglespace:
    data_dehtml['text'] = data_dehtml.text.apply(lambda x: p.sub(' ',x))
    test_dehtml['text'] = test_dehtml.text.apply(lambda x: p.sub(' ',x))



In [67]:
data_dehtml.head()

,text,label
0,DEAR SIR STRICTLY PRIVATE BUSINESS PROPOSAL AM MIKE CHUKWU THE MANAGER BILLS AND EXCHANGE AT THE FOREIGN REMITTANCE DEPARTMENT OF THE ZENITH INTERNATIONAL BANK PLC AM WRITING THIS LETTER TO ASK FOR YOUR SUPPORT AND COOPERATION TO CARRY OUT THIS BUSINESS OPPORTUNITY IN MY DEPARTMENT WE DISCOVERED AN ABANDONED SUM OF FIFTEEN MILLION UNITED STATES DOLLARS ONLY IN AN ACCOUNT THAT BELONGS TO ONE OF OUR FOREIGN CUSTOMERS WHO DIED ALONG WITH HIS ENTIRE FAMILY OF WIFE AND TWO CHILDREN IN NOVEMBER IN PLANE CRASH SINCE WE HEARD OF HIS DEATH WE HAVE BEEN EXPECTING HIS NEXTOFKIN TO COME OVER AND PUT CLAIMS FOR HIS MONEY AS THE HEIRBECAUSE WE CANNOT RELEASE THE FUND FROM HIS ACCOUNT UNLESS SOMEONE APPLIES FOR CLAIM AS THE NEXTOFKIN TO THE DECEASED AS INDICATED IN OUR BANKING GUIDELINES UNFORTUNATELY NEITHER THEIR FAMILY MEMBER NOR DISTANT RELATIVE HAS EVER APPEARED TO CLAIM THE SAID FUND UPON THIS DISCOVERY AND OTHER OFFICIALS IN MY DEPARTMENT HAVE AGREED TO MAKE BUSINESS WITH YOU AND RELEASE THE TOTAL AMOUNT INTO YOUR ACCOUNT AS THE HEIR OF THE FUND SINCE NO ONE CAME FOR IT OR DISCOVERED HE MAINTAINED ACCOUNT WITH OUR BANK OTHERWISE THE FUND WILL BE RETURNED TO THE BANKS TREASURY AS UNCLAIMED FUND WE HAVE AGREED THAT OUR RATIO OF SHARING WILL BE AS STATED THUS FOR YOU AS FOREIGN PARTNER FOR US THE OFFICIALS IN MY DEPARTMENT AND FOR THE SETTLEMENT OF ALL LOCAL AND FOREIGN EXPENCES INCURRED BY US AND YOU DURING THE COURSE OF THIS BUSINESS UPON THE SUCCESSFUL COMPLETION OF THIS TRANSFER AND ONE OF MY COLLEAGUES WILL COME TO YOUR COUNTRY AND MIND OUR SHARE IT IS FROM OUR WE INTEND TO IMPORT AGRICULTURAL MACHINERIES INTO MY COUNTRY AS WAY OF RECYCLING THE FUND TO COMMENCE THIS TRANSACTION WE REQUIRE YOU TO IMMEDIATELY INDICATE YOUR INTEREST BY RETURN MAIL AND ENCLOSE YOUR PRIVATE CONTACT TELEPHONE NUMBER FAX NUMBER FULL NAME AND ADDRESS AND YOUR DESIGNATED BANK COORDINATES TO ENABLE US FILE LETTER OF CLAIM TO THE APPROPRIATE DEPARTMENTS FOR NECESSARY APPROVALS BEFORE THE TRANSFER CAN BE MADE NOTE ALSO THIS TRANSACTION MUST BE KEPT STRICTLY CONFIDENTIAL BECAUSE OF IT NATURE LOOK FORWARD TO RECEIVING YOUR PROMPT RESPONSE REGARDS MR MIKE CHUKWU ZENITH INTERNATIONAL BANK PLC,1
1,Will do,0
2,NoraCheryl has emailed dozens of memos about Haiti to me this weekend Can you please print them and organize them inchrono order for me for my trip tomorrow or send to Lauren so she can do Thx,0
3,Dear SirMadam know that this proposal might be surprise to you but it as an emergencyIn nutshell am MR ZUMA from the republic of SouthAfrica now seeking for refuge in Dakar Senegal under theUNHCR got your contact during desperate search here in Dakar for possible transaction My late father DR ZUMA KENT WILLIAMS he was the managing director of Gold Mine company in SOUTH AFRICA But he was assinated by his business assoicate and all his properties was totally destroyed by his elder brother Who alway hate to see me and my mother with his eye that man very wicked that is why and my mother managed to escape with some of my father documents covering Million dollars which is presently deposited safely in finance firm on my name as the next of kin Meanwhilewe are saddled with the problem of securing trustworthy foriegn personality to help us transfer the money over to his country and into his possession pending our arrival to meet with him Furthermorewe only want this done this way because your country is politically stable for any profitable investment and only if you accept our proposal you will serve as the beneficiary of the fund on commencement of this proposed transactions am giving you the offers as mentioned with every confidence on your acceptance to assist us we have decided to invest of the total fund based on equity participation in your company Secondly we shall also take out for any miscelleneous expenses that may occure Conclusively wish you send me reply immediately as soon as yourecieve this proposal so that we shall arrange on how to lift this c

In [68]:
data_dehtml[data_dehtml.text.str.startswith('b')]

,text,label
130,been released,0
339,back we need to talkcalling,0
456,briefed group of wonks at Brookings tonite Superbly impressive persuasive Lowkey mastery confidence Congrats on having her on your team leading these negs,0
2553,behavior Washington Postdoc You almost certainly have seen this but just in case AM,0
2642,because we are still in Government service Let honesty and trust be ourwatchwords throughout this transaction And your Prompt reply will be highly appreciated Best regards John ObuteThis mail was sent using prowebmail,1
3456,bsp URGENT BUSINESS PROPOSALASSISTANCE cannot imagine the surprise this will bring to you but please be rest assured is with good faith and intentions from family in dare need of your assistance to help us transfer the inheritance sum of million OLLARS US into your account or company account in which God will bless you for rendering helping hand to others am Mrs Maria Banda from Sierra Leone currently residing in SouthAfrica the wife to the late James Banda got your contact details in the course of my discreet search for reliable and God fearing person from the Johannesburg Trade an Exhibition Centre and after going throug your profile decided to solicit for your kind assistance am urrently residing with my familyson and daughter here in Sout Africa as asylum seekers and the South African Fiscal Policy Law an Regulations does not allow us certain financial rights My husband was brigadier in charge of Arms and Ammunition procurement for the Sierra Leone Army before his death After his death discovered that in his will he specifically drew my attention to the sum of MILLION US which he had deposited in safe box of the Private Security Company here in Johannesburg South Africa The lives and future of my family depends on this fund as such wil be grateful if you can assist us In view of this cannot invest his fund in South Africa hence my asking you to assist me to transfer it out of South Africa for investment purposes in your country For your efforts am prepared to offer you of the total sum while will be set aside for expenses incurred during the course of this transaction an the remaining is for the family investment in your country Please note that this transaction is riskfree and the major thing ask of you is to assure me the safety of the money when transferred to your account Furthern information and arrangement will commence soon as trust confidence and good relationship is established shall be most grateful if you maintain the confidentiality require in this transaction Do not hesitate to call me on my phone for further directives or send me email to enable us commence towards the conclusion of this transaction await for your prompt response Thanks and God bless MRS MARIA BANDA For the Family Yahoo for Good Click here to donate to the Hurricane Katrina relief effort,1
3821,back we need to talk Call only my berry,0
3914,back we need to talk Can we talk now just woke up again,0


Only few examples with initial 'b', and the 'b' mostly seems to be an innocent part of a word. (Possibly double-check original formatting of ex. 298, there's an artifact at the beginning that wasn't processed properly.)


In [69]:
data.iloc[298]

text     http://www.ft.com/cms/s/5799a774-b534-11df-9af8-00144feabdc0dwp_uuid=236ce648-33fa-11da-adae-00000e2511c8print=yes.htmlFinancial Times FT.comr.OLUMNISTSJVlartin WolfObama was too cautious in fearful timesBy Martin WolfPublished: AuÃ¢ÂÂ¢ ust 31 2010 22:48 j Last updated: August 31 2010 22:48Suppose that the US presidential election of 1932 had in fact taken place in 1930 at an early stage in the GreatDepression. Suppose too that Franklin Delano Roosevelt had won then though not by the landslide of 1932. Howdifferent subsequent events might have been. The president might have watched helplessly as output and employmentcollapsed. The decades of Democratic dominance might not have happened.On such chances the wheel of history turns. But this time was different: the crisis brought Barack Obama to power closeto the beginning of the economic collapse. I (among others) then argued that policy needed to be hugely aggressive. Alasit was not. I noted on February 4 2009 at the beginning 

In [70]:
data_dehtml.iloc[298]

text      Financial Times FTcomr OLUMNISTSJVlartin Wolf Obama was too cautious in fearful times By Martin Wolf Published Au ust Last updated August Suppose that the US presidential election of had in fact taken place in at an early stage in the Great Depression Suppose too that Franklin Delano Roosevelt had won then though not by the landslide of Howdifferent subsequent events might have been The president might have watched helplessly as output and employmentcollapsed The decades of Democratic dominance might not have happened On such chances the wheel of history turns But this time was different the crisis brought Barack Obama to power closeto the beginning of the economic collapse among others then argued that policy needed to be hugely aggressive Alasit was not noted on February at the beginning of the new presidency Instead of an overwhelming fiscalstimulus what is emerging is too small too wasteful and too illfocused week later asked Has Barack Obamaspresidency already failed In 

Ok, should add '-' and '=' and '&' to the website regex.

To be safe (as it's specifically mentioned in the instructions), also check 'b' directly preceding a capital letter (assuming that would be a case of prefixing?):

In [71]:
# only uncomment if you want to see a really long text field of letter garbage
# initb = re.compile(r'\s([b][A-Z])')

# data_dehtml[data_dehtml.text.apply(lambda x: True if initb.search(x) != None else False)]

Only one example with 'b' prefixed directly to a capital letter in this spam example. Leaving this message intact, probably just a clear instance of spam.

In [72]:
# Your code
data_dehtml['text'] = data_dehtml.text.str.lower()
test_dehtml['text'] = test_dehtml.text.str.lower()

In [73]:
data_dehtml.head()

,text,label
0,dear sir strictly private business proposal am mike chukwu the manager bills and exchange at the foreign remittance department of the zenith international bank plc am writing this letter to ask for your support and cooperation to carry out this business opportunity in my department we discovered an abandoned sum of fifteen million united states dollars only in an account that belongs to one of our foreign customers who died along with his entire family of wife and two children in november in plane crash since we heard of his death we have been expecting his nextofkin to come over and put claims for his money as the heirbecause we cannot release the fund from his account unless someone applies for claim as the nextofkin to the deceased as indicated in our banking guidelines unfortunately neither their family member nor distant relative has ever appeared to claim the said fund upon this discovery and other officials in my department have agreed to make business with you and release the total amount into your account as the heir of the fund since no one came for it or discovered he maintained account with our bank otherwise the fund will be returned to the banks treasury as unclaimed fund we have agreed that our ratio of sharing will be as stated thus for you as foreign partner for us the officials in my department and for the settlement of all local and foreign expences incurred by us and you during the course of this business upon the successful completion of this transfer and one of my colleagues will come to your country and mind our share it is from our we intend to import agricultural machineries into my country as way of recycling the fund to commence this transaction we require you to immediately indicate your interest by return mail and enclose your private contact telephone number fax number full name and address and your designated bank coordinates to enable us file letter of claim to the appropriate departments for necessary approvals before the transfer can be made note also this transaction must be kept strictly confidential because of it nature look forward to receiving your prompt response regards mr mike chukwu zenith international bank plc,1
1,will do,0
2,noracheryl has emailed dozens of memos about haiti to me this weekend can you please print them and organize them inchrono order for me for my trip tomorrow or send to lauren so she can do thx,0
3,dear sirmadam know that this proposal might be surprise to you but it as an emergencyin nutshell am mr zuma from the republic of southafrica now seeking for refuge in dakar senegal under theunhcr got your contact during desperate search here in dakar for possible transaction my late father dr zuma kent williams he was the managing director of gold mine company in south africa but he was assinated by his business assoicate and all his properties was totally destroyed by his elder brother who alway hate to see me and my mother with his eye that man very wicked that is why and my mother managed to escape with some of my father documents covering million dollars which is presently deposited safely in finance firm on my name as the next of kin meanwhilewe are saddled with the problem of securing trustworthy foriegn personality to help us transfer the money over to his country and into his possession pending our arrival to meet with him furthermorewe only want this done this way because your country is politically stable for any profitable investment and only if you accept our proposal you will serve as the beneficiary of the fund on commencement of this proposed transactions am giving you the offers as mentioned with every confidence on your acceptance to assist us we have decided to invest of the total fund based on equity participation in your company secondly we shall also take out for any miscelleneous expenses that may occure conclusively wish you send me reply immediately as soon as yourecieve this proposal so that we shall arrange on how to lift this c

### Let's tokenize

In [74]:
from nltk import word_tokenize

data_token = data_dehtml.copy()
test_token = test_dehtml.copy()

data_token['text'] = data_token.text.apply(word_tokenize)
test_token['text'] = test_token.text.apply(word_tokenize)

In [75]:
data_token.head()

,text,label
0,"[dear, sir, strictly, private, business, proposal, am, mike, chukwu, the, manager, bills, and, exchange, at, the, foreign, remittance, department, of, the, zenith, international, bank, plc, am, writing, this, letter, to, ask, for, your, support, and, cooperation, to, carry, out, this, business, opportunity, in, my, department, we, discovered, an, abandoned, sum, of, fifteen, million, united, states, dollars, only, in, an, account, that, belongs, to, one, of, our, foreign, customers, who, died, along, with, his, entire, family, of, wife, and, two, children, in, november, in, plane, crash, since, we, heard, of, his, death, we, have, been, expecting, his, nextofkin, to, come, over, ...]",1
1,"[will, do]",0
2,"[noracheryl, has, emailed, dozens, of, memos, about, haiti, to, me, this, weekend, can, you, please, print, them, and, organize, them, inchrono, order, for, me, for, my, trip, tomorrow, or, send, to, lauren, so, she, can, do, thx]",0
3,"[dear, sirmadam, know, that, this, proposal, might, be, surprise, to, you, but, it, as, an, emergencyin, nutshell, am, mr, zuma, from, the, republic, of, southafrica, now, seeking, for, refuge, in, dakar, senegal, under, theunhcr, got, your, contact, during, desperate, search, here, in, dakar, for, possible, transaction, my, late, father, dr, zuma, kent, williams, he, was, the, managing, director, of, gold, mine, company, in, south, africa, but, he, was, assinated, by, his, business, assoicate, and, all, his, properties, was, totally, destroyed, by, his, elder, brother, who, alway, hate, to, see, me, and, my, mother, with, his, eye, that, man, very, wicked, ...]",1
4,[fyi],0


## Now let's work on removing stopwords
Remove the stopwords.

In [76]:
# Your code
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

data_token['text'] = data_token.text.apply(lambda input: list(filter(lambda x: '' if x in stop_words else x, input)))
test_token['text'] = test_token.text.apply(lambda input: list(filter(lambda x: '' if x in stop_words else x, input)))

In [77]:
data_token.head()

,text,label
0,"[dear, sir, strictly, private, business, proposal, mike, chukwu, manager, bills, exchange, foreign, remittance, department, zenith, international, bank, plc, writing, letter, ask, support, cooperation, carry, business, opportunity, department, discovered, abandoned, sum, fifteen, million, united, states, dollars, account, belongs, one, foreign, customers, died, along, entire, family, wife, two, children, november, plane, crash, since, heard, death, expecting, nextofkin, come, put, claims, money, heirbecause, release, fund, account, unless, someone, applies, claim, nextofkin, deceased, indicated, banking, guidelines, unfortunately, neither, family, member, distant, relative, ever, appeared, claim, said, fund, upon, discovery, officials, department, agreed, make, business, release, total, amount, account, heir, fund, since, one, came, discovered, ...]",1
1,[],0
2,"[noracheryl, emailed, dozens, memos, haiti, weekend, please, print, organize, inchrono, order, trip, tomorrow, send, lauren, thx]",0
3,"[dear, sirmadam, know, proposal, might, surprise, emergencyin, nutshell, mr, zuma, republic, southafrica, seeking, refuge, dakar, senegal, theunhcr, got, contact, desperate, search, dakar, possible, transaction, late, father, dr, zuma, kent, williams, managing, director, gold, mine, company, south, africa, assinated, business, assoicate, properties, totally, destroyed, elder, brother, alway, hate, see, mother, eye, man, wicked, mother, managed, escape, father, documents, covering, million, dollars, presently, deposited, safely, finance, firm, name, next, kin, meanwhilewe, saddled, problem, securing, trustworthy, foriegn, personality, help, us, transfer, money, country, possession, pending, arrival, meet, furthermorewe, want, done, way, country, politically, stable, profitable, investment, accept, proposal, serve, beneficiary, fund, commencement, proposed, ...]",1
4,[fyi],0


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [78]:
from nltk.corpus import wordnet


In [79]:
# Your code
from nltk.stem import WordNetLemmatizer

wordnet_lemma  = WordNetLemmatizer()


def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = pos_tag([word])[0][1][0]#  Get POS tag's first character (e.g., 'N' from 'NN')
    #Maps it to a WordNet-compatible tag
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    
    return tag_dict.get(tag, wordnet.NOUN) # returns the word type (Noun if we have not found)
    
def lemmatize(lst):
    return [wordnet_lemma.lemmatize(word,pos=get_wordnet_pos(word)) for word in lst]

bow = ['this', 'is', 'a', 'beautiful', 'set', 'of', 'examples', 'blsds']

In [80]:

data_lemma = data_token.copy()
test_lemma = test_token.copy()

data_lemma['text'] = data_lemma.text.apply(lemmatize)
test_lemma['text'] = test_lemma.text.apply(lemmatize)

In [81]:
data_lemma.head()

,text,label
0,"[dear, sir, strictly, private, business, proposal, mike, chukwu, manager, bill, exchange, foreign, remittance, department, zenith, international, bank, plc, write, letter, ask, support, cooperation, carry, business, opportunity, department, discover, abandon, sum, fifteen, million, united, state, dollar, account, belongs, one, foreign, customer, die, along, entire, family, wife, two, child, november, plane, crash, since, heard, death, expect, nextofkin, come, put, claim, money, heirbecause, release, fund, account, unless, someone, applies, claim, nextofkin, decease, indicate, banking, guideline, unfortunately, neither, family, member, distant, relative, ever, appear, claim, say, fund, upon, discovery, official, department, agree, make, business, release, total, amount, account, heir, fund, since, one, come, discover, ...]",1
1,[],0
2,"[noracheryl, email, dozen, memo, haiti, weekend, please, print, organize, inchrono, order, trip, tomorrow, send, lauren, thx]",0
3,"[dear, sirmadam, know, proposal, might, surprise, emergencyin, nutshell, mr, zuma, republic, southafrica, seek, refuge, dakar, senegal, theunhcr, get, contact, desperate, search, dakar, possible, transaction, late, father, dr, zuma, kent, williams, manage, director, gold, mine, company, south, africa, assinated, business, assoicate, property, totally, destroyed, elder, brother, alway, hate, see, mother, eye, man, wicked, mother, manage, escape, father, document, cover, million, dollar, presently, deposit, safely, finance, firm, name, next, kin, meanwhilewe, saddle, problem, secure, trustworthy, foriegn, personality, help, u, transfer, money, country, possession, pending, arrival, meet, furthermorewe, want, do, way, country, politically, stable, profitable, investment, accept, proposal, serve, beneficiary, fund, commencement, propose, ...]",1
4,[fyi],0


In [82]:
# rejoin the lists into single strings to get appropriate input for vectorisers

data_lemma['text'] = data_lemma.text.apply(lambda x: ' '.join(x))
test_lemma['text'] = test_lemma.text.apply(lambda x: ' '.join(x))

In [85]:
test_lemma.head()

,text
0,usiness fact decease man foreigner authorize law guide bank citizen country make claim fund request foreigner come business necessary facilitate release transfer fund smoothly reliable bank account survive relation decease really sure integrity trustworthy confidentiality accept cheat disappoint afte business reply telephone fax number banking account detail wish receive fund upon receipt mention information indicate interest business send fax mail letter application type send back bank enable u start transaction fail bring notice business risk free negative implication entertain fear odalities smooth easy transfer fund finalize transaction complete within bank work day immediatel start business wait urgent response please kindly delete interested regard mr
1,happy adjust afternoon go suggest pm start time give plenty time toreturn dc cronkite service start
2,lael brainard confirm afternoon miguel rodriguez deputy assistant secretary senate affair bureau legislative affair department state
3,friday march amsbwhoeop extend congrats
4,dear good friend happy inform successin get fund transfer cooperation ne partner hongkong presently hongkong investment project share total sum meanwhile ididn forget past effort attempt assist transfer fund despite failedus contact secretar cotonou benin republic hisname mr shikes umahand emailsecre ask send total kept past effort attempt toassist matter appreciate effort atthat time ver much feel free get intouchedwith secretary shikes umah instruct sendthe amount please le know immediately receive sothat share joy al sufferness time moment busy becauseof investment project new partner hand finally remember ihad forward instruction secretary behalf receive money feel free et intouch mr shikes umah send amount delay thanks god bless shikes umah mikejames james


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [91]:
X = data_lemma['text'].values
y = data_lemma['label'].values

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=8)

extratest = test_lemma['text'].values

In [92]:


# Your code
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)
extratest_bow = vectorizer.transform(extratest)

feature_names = vectorizer.get_feature_names_out()

# Separate texts by label
label_0_indices = np.where(y_train == 0)[0]
label_1_indices = np.where(y_train == 1)[0]

# Sum word counts per label
label_0_counts = X_train_bow[label_0_indices].sum(axis=0)
label_1_counts = X_train_bow[label_1_indices].sum(axis=0)

# Convert to 1D array
label_0_counts = np.asarray(label_0_counts).flatten()
label_1_counts = np.asarray(label_1_counts).flatten()

# Get top 10 words per label
top_10_label_0 = np.argsort(label_0_counts)[::-1][:10]
top_10_label_1 = np.argsort(label_1_counts)[::-1][:10]

# Display results
print("Top 10 words for ham:")
for idx in top_10_label_0:
    print(f"{feature_names[idx]}: {label_0_counts[idx]}")

print("\nTop 10 words for spam:")
for idx in top_10_label_1:
    print(f"{feature_names[idx]}: {label_1_counts[idx]}")



Top 10 words for ham:
state: 998
say: 890
pm: 682
call: 604
would: 572
secretary: 557
work: 535
obama: 508
time: 500
president: 490

Top 10 words for spam:
money: 4449
account: 3838
bank: 3784
fund: 3461
transfer: 2314
transaction: 2245
business: 2115
country: 2100
contact: 2043
mr: 1941


In [87]:
X_train_bow.shape

(5964, 91048)

## Extra features

In [88]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data['money_mark'] = data_lemma['text'].str.contains(money_simbol_list)*1
data['suspicious_words'] = data_lemma['text'].str.contains(suspicious_words)*1
data['text_len'] = data_lemma['text'].apply(lambda x: len(x)) 

# what is data_val?
# data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
# data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
# data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data.head()

,text,label,money_mark,suspicious_words,text_len
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL I AM MIKE CHUKWU , THE MANAGER, BILLS AND EXCHANGE AT THE FOREIGN REMITTANCE DEPARTMENT OF THE ZENITH INTERNATIONAL BANK PLC. I AM WRITING THIS LETTER TO ASK FOR YOUR SUPPORT AND COOPERATION TO CARRY OUT THIS BUSINESS OPPORTUNITY IN MY DEPARTMENT. WE DISCOVERED AN ABANDONED SUM OF $15,000,000.00 (FIFTEEN MILLION UNITED STATES DOLLARS ONLY) IN AN ACCOUNT THAT BELONGS TO ONE OF OUR FOREIGN CUSTOMERS WHO DIED ALONG WITH HIS ENTIRE FAMILY OF A WIFE AND TWO CHILDREN IN NOVEMBER 1997 IN A PLANE CRASH. SINCE WE HEARD OF HIS DEATH, WE HAVE BEEN EXPECTING HIS NEXT-OF-KIN TO COME OVER AND PUT CLAIMS FOR HIS MONEY AS THE HEIR,BECAUSE WE CANNOT RELEASE THE FUND FROM HIS ACCOUNT UNLESS SOMEONE APPLIES FOR CLAIM AS THE NEXT-OF-KIN TO THE DECEASED AS INDICATED IN OUR BANKING GUIDELINES. UNFORTUNATELY, NEITHER THEIR FAMILY MEMBER NOR DISTANT RELATIVE HAS EVER APPEARED TO CLAIM THE SAID FUND. UPON THIS DISCOVERY,I AND OTHER OFFICIALS IN MY DEPARTMENT HAVE AGREED TO MAKE BUSINESS WITH YOU AND RELEASE THE TOTAL AMOUNT INTO YOUR ACCOUNT AS THE HEIR OF THE FUND SINCE NO ONE CAME FOR IT OR DISCOVERED HE MAINTAINED ACCOUNT WITH OUR BANK, OTHERWISE THE FUND WILL BE RETURNED TO THE BANKS TREASURY AS UNCLAIMED FUND. WE HAVE AGREED THAT OUR RATIO OF SHARING WILL BE AS STATED THUS; 30 % FOR YOU AS FOREIGN PARTNER, 60 % FOR US THE OFFICIALS IN MY DEPARTMENT AND 10% FOR THE SETTLEMENT OF ALL LOCAL AND FOREIGN EXPENCES INCURRED BY US AND YOU DURING THE COURSE OF THIS BUSINESS. UPON THE SUCCESSFUL COMPLETION OF THIS TRANSFER, I AND ONE OF MY COLLEAGUES WILL COME TO YOUR COUNTRY AND MIND OUR SHARE. IT IS FROM OUR 60 % WE INTEND TO IMPORT AGRICULTURAL MACHINERIES INTO MY COUNTRY AS A WAY OF RECYCLING THE FUND. TO COMMENCE THIS TRANSACTION, WE REQUIRE YOU TO IMMEDIATELY INDICATE YOUR INTEREST BY A RETURN E-MAIL AND ENCLOSE YOUR PRIVATE CONTACT TELEPHONE NUMBER, FAX NUMBER FULL NAME AND ADDRESS AND YOUR DESIGNATED BANK COORDINATES TO ENABLE US FILE LETTER OF CLAIM TO THE APPROPRIATE DEPARTMENTS FOR NECESSARY APPROVALS BEFORE THE TRANSFER CAN BE MADE. NOTE ALSO, THIS TRANSACTION MUST BE KEPT STRICTLY CONFIDENTIAL BECAUSE OF IT'S NATURE. I LOOK FORWARD TO RECEIVING YOUR PROMPT RESPONSE. REGARDS, MR. MIKE CHUKWU ZENITH INTERNATIONAL BANK PLC .",1,1,1,1461
1,Will do.,0,1,0,0
2,Nora--Cheryl has emailed dozens of memos about Haiti to me this weekend. Can you please print them and organize them inchrono order for me for my trip tomorrow or send to Lauren so she can do? Thx.,0,1,0,108
3,Dear Sir=2FMadam=2C I know that this proposal might be a surprise to you but it as an emergency=2EIn a nutshell=2C I am MR=2EP=2E ZUMA=2C from the republic of South-Africa=2C now seeking for refuge in Dakar Senegal under the=28UNHCR=29=2E I got your contact during a desperate search here in Dakar for a possible transaction=2E My =28late=29 father DR=2E ZUMA KENT WILLIAMS=2C he was the managing director of Gold Mine company in SOUTH AFRICA=2E But he was assinated by his business assoicate and all his properties was totally destroyed by his elder brother=2E Who alway hate to see me and my mother with his eye's=2C that man very wicked that is why i and my mother managed to escape with some of my father's documents covering $18=2E3 Million dollars which is presently deposited safely in a finance firm on my name as the next of kin=2E Meanwhile=2Cwe are saddled with the problem of securing a trust-worthy foriegn personality to help us transfer the money over to his country and into his possession pending our arrival to meet with him=2E Furthermore=2Cwe only want this done this way because your country is politically stable for any profitable investment and only if you accept our proposal=2C you will serve as the beneficiary of the fund on commencement of this proposed transactions=2E I am giving you the offers as mentioned with every confidence on your acceptance to assist us=2C we have decided t

## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code
# I'm not sure I understand the question in this context? I fit the countVectorizer above

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [93]:
# Your code

tfidfvectorizer = TfidfVectorizer()
tfidf_matrix = tfidfvectorizer.fit_transform(X_train)
feature_names = tfidfvectorizer.get_feature_names_out()
X_train_tfidf = tfidf_matrix.toarray()

X_train_tfidf.shape


(4771, 73088)

## And the Train a Classifier?

In [99]:
# Your code

X_test_tfidf = tfidfvectorizer.transform(X_test).toarray()

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix

logreg = LogisticRegression(random_state=6)
logreg.fit(X_train_tfidf,y_train)

y_pred = logreg.predict(X_test_tfidf)







In [100]:

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[657   2]
 [ 43 491]]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       659
           1       1.00      0.92      0.96       534

    accuracy                           0.96      1193
   macro avg       0.97      0.96      0.96      1193
weighted avg       0.96      0.96      0.96      1193



In [102]:
# the result is quite decent with 96% accuracy
# now we could also make predictions for the unlabeled test set

extratest_tfidf = tfidfvectorizer.transform(extratest).toarray()
testresults = logreg.predict(extratest_tfidf)
testresults

array([1, 0, 0, ..., 0, 0, 0])

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code